# Applied Data Science Capstone - _Analysis of the Restaurant Market in South Mumbai_

<h3> Data Section</h3>

* importing libraries

In [2]:
!pip install geocoder

from bs4 import BeautifulSoup
import requests
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import urllib
import geocoder 


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

    100% |████████████████████████████████| 102kB 16.2MB/s 
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction

* Fetching data from CSV file downloaded from Open Government Data (OGD) Platform India (Digital India Initiative)

In [3]:
url = "mumbai_neighbourhoods.csv"
mumbai_data = pd.read_csv(url)
mumbai_data.tail()

,Neighbourhood,Pincode,officeType,Deliverystatus,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Borough,longitude,latitude
1002,Wada,421303,NaN,Delivery,Mumbai,Maharashtra,Wada,Thane,MAHARASHTRA,02526-271413,NaN,Palghar,NaN,NaN
1003,Waki B.O,401602,B.O,Delivery,Mumbai,Maharashtra,Dahanu,Thane,MAHARASHTRA,NaN,Dahanu Road,Palghar,NaN,NaN
1004,Waki B.O,421303,B.O,Delivery,Mumbai,Maharashtra,Vikramgad,Thane,MAHARASHTRA,NaN,Wada,Palghar,NaN,NaN
1005,Zap B.O,401603,B.O,Delivery,Mumbai,Maharashtra,Jawhar,Thane,MAHARASHTRA,NaN,Jawhar,Palghar,NaN,NaN
1006,Zari B.O,401606,B.O,Delivery,Mumbai,Maharashtra,Talasari,Thane,MAHARASHTRA,NaN,Talasari,Palghar,NaN,NaN


* Project is focus on 'South Mumbai' so data filtering as per requirement

In [6]:
mumbai_data = mumbai_data[mumbai_data.Borough=='South Mumbai']
mumbai_data

,Neighbourhood,Pincode,officeType,Deliverystatus,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Borough,longitude,latitude
0,B.P.Lane,400003,NaN,Non-Delivery,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23421653,NaN,South Mumbai,NaN,NaN
1,BEST STaff Quarters,400012,NaN,Non-Delivery,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-24180776,NaN,South Mumbai,NaN,NaN
2,Chamarbaug,400012,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24705407,NaN,South Mumbai,NaN,NaN
3,Chinchbunder,400009,NaN,Delivery,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23771116,NaN,South Mumbai,NaN,NaN
4,Cotton Exchange,400033,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-23742035,NaN,South Mumbai,NaN,NaN
5,Dockyard Road,400010,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-23774238,NaN,South Mumbai,NaN,NaN
6,Kalachowki,400033,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-23724722,NaN,South Mumbai,NaN,NaN
7,L B S N E collage,400033,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-23744235,NaN,South Mumbai,NaN,NaN
8,Lal Baug,400012,NaN,Non-Delivery,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24702340,NaN,South Mumbai,NaN,NaN
9,Mandvi (Mumbai),400003,NaN,Delivery,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23432740,NaN,South Mumbai,NaN,NaN


* Data cleaning as removing redundancy and unwanted duplicity

1.Join all the AreaName by Pincode

In [7]:
def neighbourhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = mumbai_data.groupby(['Pincode', 'Borough'])
south_mumbai_data = grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')

2.Displaying south_mumbai_data 

In [8]:
south_mumbai_data

,Pincode,Borough,Neighbourhood
0,400001,South Mumbai,"Bazargate , CSMT , M.P.T. , Stock Exchange , T..."
1,400002,South Mumbai,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar"
2,400003,South Mumbai,"B.P.Lane , Mandvi (Mumbai), Masjid , Null Bazar"
3,400004,South Mumbai,"Ambewadi (Mumbai), Charni Road , Chaupati , G..."
4,400005,South Mumbai,"Asvini , Colaba , Colaba Bazar , Holiday Camp ..."
5,400006,South Mumbai,Malabar Hill
6,400007,South Mumbai,"Bharat Nagar (Mumbai), Grant Road , N.S.Patka..."
7,400008,South Mumbai,"Falkland Road , J.J.Hospital , Kamathipura , M..."
8,400009,South Mumbai,"Chinchbunder , Noor Baug , Princess Dock"
9,400010,South Mumbai,"Dockyard Road , Mazgaon , Mazgaon Dock , Mazga..."


3.Getting Coordinates as per pincode

In [9]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, Maharashtra'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

e.g, For Pincode 400001

In [10]:
get_latlng('400001')

[18.938919252000062, 72.83747000000005]

Now appling this functions for all pincode of areas of South Mumbai

In [12]:
latitude = []
longitude = []
for row in south_mumbai_data['Pincode']:
    coordinate = get_latlng(row)
    latitude.append(coordinate[0])
    longitude.append(coordinate[1])
    
south_mumbai_data['Latitude']=latitude
south_mumbai_data['Longitude']=longitude
south_mumbai_data    

,Pincode,Borough,Neighbourhood,Latitude,Longitude
0,400001,South Mumbai,"Bazargate , CSMT , M.P.T. , Stock Exchange , T...",18.938919,72.837470
1,400002,South Mumbai,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar",18.947760,72.826815
2,400003,South Mumbai,"B.P.Lane , Mandvi (Mumbai), Masjid , Null Bazar",18.951733,72.835129
3,400004,South Mumbai,"Ambewadi (Mumbai), Charni Road , Chaupati , G...",18.955285,72.820930
4,400005,South Mumbai,"Asvini , Colaba , Colaba Bazar , Holiday Camp ...",18.904399,72.810195
5,400006,South Mumbai,Malabar Hill,18.952990,72.799974
6,400007,South Mumbai,"Bharat Nagar (Mumbai), Grant Road , N.S.Patka...",18.961125,72.818175
7,400008,South Mumbai,"Falkland Road , J.J.Hospital , Kamathipura , M...",18.968163,72.826718
8,400009,South Mumbai,"Chinchbunder , Noor Baug , Princess Dock",18.956600,72.842166
9,400010,South Mumbai,"Dockyard Road , Mazgaon , Mazgaon Dock , Mazga...",18.972625,72.844159


* Loading property data which are researched from different Indian propert portals and property news and professional realtors which stored in CSV file

In [13]:
property_url = 'South_Mumbai_Real_Estate.csv'
property_data = pd.read_csv(property_url) 
property_data

,Pincode,Neighbourhood,Avg. Price Range (Rs. per sq. ft.)
0,400001,"Bazargate , CSMT , M.P.T. , Stock Exchange , T...",68817.0
1,400002,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar",49502.0
2,400003,"B.P.Lane , Mandvi (Mumbai), Masjid , Null Bazar",16666.0
3,400004,"Ambewadi (Mumbai), Charni Road , Chaupati , G...",32926.0
4,400005,"Asvini , Colaba , Colaba Bazar , Holiday Camp ...",68817.0
5,400006,Malabar Hill,125761.0
6,400007,"Bharat Nagar (Mumbai), Grant Road , N.S.Patka...",48103.0
7,400008,"Falkland Road , J.J.Hospital , Kamathipura , M...",38210.0
8,400009,"Chinchbunder , Noor Baug , Princess Dock",16667.0
9,400010,"Dockyard Road , Mazgaon , Mazgaon Dock , Mazga...",17021.0


* South Mumbai ward wise average property price

In [27]:
southmumbai_propertyrate = 'BMCWardproperties.csv'
southmumbai_propertyrate_data = pd.read_csv(southmumbai_propertyrate) 
southmumbai_propertyrate_data

,Ward,Avg Price
0,A,125761
1,B,45900
2,C,65800
3,D,105753
4,E,55800
5,F,45000
6,G,50500
7,H,30000
8,I,40000


* Fetching details from wikipedia page of Administrative divisions of Mumbai
1.Fetch data from wikipedia

In [26]:
# fetching data from wikipedia page and storing into m_wiki_data
url = "https://en.wikipedia.org/wiki/Administrative_divisions_of_Mumbai"
m_wiki_data = requests.get(url).text
interm = m_wiki_data[m_wiki_data.find("<table"):m_wiki_data.find("</table>")+8]
df = pd.read_html(interm, header = 0)[0]
df

,Ward,Population
0,A,210926
1,B,140480
2,C,190670
3,D,378610
4,E,439390
5,F,430406
6,G,537032
7,H,478413
8,I,516552


* Next, we are going to start utilizing the Foursquare API to explore the South Mumbabi for data collection

In [50]:
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXX
CLIENT_SECRET:XXXXX


1.Taking first area of South Mumbai and creating url

In [43]:
first_lat = south_mumbai_data['Latitude'][0]
first_long = south_mumbai_data['Longitude'][0]
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_lat, 
    first_long, 
    radius, 
    LIMIT)

2.Getting results

In [44]:
results = requests.get(url).json()

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

3.Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sher-E-Punjab,Indian Restaurant,18.937944,72.837853
1,Chhatrapati Shivaji Maharaj Terminus,Train Station,18.940297,72.835384
2,Royal China,Chinese Restaurant,18.938715,72.832933
3,Dakshin Bar And Kitchen,Seafood Restaurant,18.936489,72.837490
4,Town House Cafe,Bar,18.938550,72.833464


This way we are gathering data from foursquare API

We have completed below data collection process.

1.	Open Government Data (OGD) Platform India (Digital India Initiative) ( All India Pin code directory)
--CSV file import

2.	Municipal Corporation of Greater Mumbai (Mumbai ward distribution data) 
--CSV file import

3.	Research of property rate from different Property websites (Moneycontrol.com, 99acres.com, MagicBricks.com, Housing.com ) and different realtors to collect the data manually
--CSV file import

4.	Foursquare API (Restaurants details of South Mumbai)
https://developer.foursquare.com/
--Calling foursquare api and collecting data

5.	GeoJSON (for representing simple geographical features)
--Calling for finding longitude and latitude of location via pincode

6. Administrative divisions of Mumbai - Wikipedia (For Population of South Mumbai)
https://en.wikipedia.org/wiki/Administrative_divisions_of_Mumbai
--Getting data from wikipedia and storing into our datafraame


<h2>Completion of Data Collection part</h2>

"Thank you for reading"